In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [ ]:
# import IPython
# import sys
# from pygaggle.rerank.transformer import MonoT5
# !{sys.executable} -m pip install -r pygaggle/requirements.txt
# IPython.Application.instance().kernel.do_shutdown(True)  

In [ ]:
# """ Generate Taskgraphs from datasets """

# from models_datasets.recipe_1m_model import AbstractModelDataset
# from models_datasets.recipe_1m_model import Recipe1MModel
# from models_datasets.wikihow_model import WikihowModel

# dataset_models = {
#     "COOKING" : Recipe1MModel,
#     "DIY": WikihowModel,
# }

# for model_name, model in dataset_models.items():
#     dataset_model: AbstractModelDataset = model()
#     print(f"Dataset Model: {model_name}")
#     dataset_model.convert_to_taskgraphs()

In [2]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()

,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [3]:
import numpy as np
import pandas as pd
from models_indexes.bm25_grid_search import GridSearchCV

# model = Bm25ModelGridCV(domain = "COOKING", queries = queries["COOKING"])
# model.set_k1_range(0.2,1, 0.2)
# model.set_b_range(0.1, 0.3, 0.1)
# model.fit()

# Load annotated data
annotated_data = pd.read_csv('/home/ubuntu/task-search-quality/queries/cooking.csv')

# # Split annotated data into training and test sets
# train_data = annotated_data.sample(frac=0.8, random_state=42)
# test_data = annotated_data.drop(train_data.index)

# Define the parameter grid to search over
param_grid = {
    'k1': np.arange(0.1, 5.0, 0.1),
    'b': np.arange(0.1, 1.0, 0.1),
}

# Define the GridSearchCV object
grid_search = GridSearchCV(domain = "COOKING", queries = queries["COOKING"], params = param_grid, cv=5)

grid_search.predict()
# # Print the best hyperparameters and score on the test set
# print("Best hyperparameters:", grid_search.best_params_)
# print("Mean average precision (MAP) score on test set:", grid_search.score(test_data['query'], test_data['doc_id']))


Generating run... k1=0.9, b=0.1
MAP scores: [0.17821700175449945, 0.18493430938656402, 0.17251592434915675, 0.1752021001556227]
Generating run... k1=0.9, b=0.2
MAP scores: [0.18908518276407432, 0.1936739228216466, 0.18369057185317894, 0.18414963335054654]
Generating run... k1=0.9, b=0.30000000000000004
MAP scores: [0.19516377591824827, 0.2005460421289325, 0.1916787664261329, 0.19117975645797453]
Generating run... k1=0.9, b=0.4
MAP scores: [0.19793983841078994, 0.20231345850394053, 0.19562495918116735, 0.1920367413149909]
Generating run... k1=0.9, b=0.5
MAP scores: [0.19380062819351346, 0.20162175730361023, 0.19428824804758302, 0.18884347661945888]
Generating run... k1=0.9, b=0.6
MAP scores: [0.18662895634539683, 0.19538604830745115, 0.18736155262839851, 0.18316091583330346]
Generating run... k1=0.9, b=0.7000000000000001
MAP scores: [0.17798990543433035, 0.18721593629187175, 0.17866790467254204, 0.17518340164312793]
Generating run... k1=0.9, b=0.8
MAP scores: [0.167858705324791, 0.17637

(array([[0.178217  , 0.18493431, 0.17251592, 0.1752021 ],
        [0.18908518, 0.19367392, 0.18369057, 0.18414963],
        [0.19516378, 0.20054604, 0.19167877, 0.19117976],
        [0.19793984, 0.20231346, 0.19562496, 0.19203674],
        [0.19380063, 0.20162176, 0.19428825, 0.18884348],
        [0.18662896, 0.19538605, 0.18736155, 0.18316092],
        [0.17798991, 0.18721594, 0.1786679 , 0.1751834 ],
        [0.16785871, 0.17637799, 0.16828296, 0.16376894],
        [0.15524755, 0.16380947, 0.15607838, 0.15131938]]),
 [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9])

In [ ]:
grid_search.split_queries

In [ ]:
# from models_indexes.bm25_model import BM25Model
# from models_indexes.ance_model import AnceModel
# from models_indexes.colbert_model import ColbertModel
from models_indexes.marqo_model import MarqoModel
# from models_indexes.abstract_model import AbstractModel
# from models_indexes.hybrid_model import HybridModel

models = {}

# for domain in  ["DIY"]:
for domain in  ["DIY", "COOKING"]:
    print(domain)
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance+t5": AnceModel(domain = domain, t5=True),
        # "colbert+t5": ColbertModel(domain = domain, t5=True),
        "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain, doc_build="separate"),
        # "hybrid": HybridModel(domain = domain, rm3=True, t5=True)
    }

In [ ]:
            # "_id": taskmap.taskmap_id,
            # "Title": taskmap.title,
            # "Date": taskmap.date,
            # "Steps": self.__parse_steps(taskmap),
            # "Tags": self.__parse_tags(taskmap),
            # "Requirements": self.__parse_requirements(taskmap),
            # "Domain": taskmap.domain_name,
            # "Description": taskmap.description,
from models_indexes.marqo_model import MarqoModel
model = MarqoModel(domain = "COOKING", t5 = False)
qs = queries["COOKING"]

# # model.convert_search_results_to_run(qs)
# resp1 = model.search("pasta")
# resp2 = model.search("fix kitchen",["Title"])
# resp2

# model.convert_search_results_to_run_attributes(qs, filters=["Tags"])
# model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps"])
# model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps", "Title"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Requirements"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps" ,"Requirements"])
model.convert_search_results_to_run_attributes(qs, filters=[ "Steps" ,"Requirements"])
# model.convert_search_results_to_run_attributes(qs, filters=["Steps"])
# model.convert_search_results_to_run_attributes(qs, filters=["Steps", "Title"])
# model.convert_search_results_to_run_attributes(qs, filters=["Title"])



In [ ]:
from models_indexes.bm25_model import BM25Model

model = BM25Model(domain = "DIY")

model.search_best("clean bathroom")

In [ ]:
resp2
# build indexes
# print("Build indexes")
# for domain, index_models in models.items():
#     for model_name, model in index_models.items():
#         model.build_index()

In [ ]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)